In [6]:
import pandas as pd
import gspread
from google.oauth2 import service_account
import sqlite3

# Google Sheets API および Google Drive API のスコープを指定
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'C:\\Users\\udano\\OneDrive\\デスクトップ\\Tech0\\Step3-1\\SUUMO\\googlespreadsheet.json'

# 認証情報を用いてGoogleSheetsにアクセス
gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)

# 対象のスプレッドシートとワークシートを指定
SPREADSHEET_KEY = '1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4'
sheet1 = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート1")
sheet2 = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート2")

# Google スプレッドシートからデータを読み込む
data_sheet1 = sheet1.get_all_values()
data_sheet2 = sheet2.get_all_values()

# データフレームに変換
df_sheet1 = pd.DataFrame(data_sheet1[1:], columns=data_sheet1[0])
df_sheet2 = pd.DataFrame(data_sheet2[1:], columns=data_sheet2[0])

# シート1に 'charge' 列がない場合のみ追加（NaNで埋める）
if 'charge' not in df_sheet1.columns:
    df_sheet1['charge'] = pd.NA

# データの整形（共通の列を保持）
common_columns = list(set(df_sheet1.columns) & set(df_sheet2.columns))
df_sheet1 = df_sheet1[common_columns]
df_sheet2 = df_sheet2[common_columns]

# データをマージする
merged_df = pd.concat([df_sheet1, df_sheet2])

# 列の順序を指定
column_order = ['title', 'address', 'age', 'floor', 'fee', 'management_fee', 'deposit', 'gratuity', 'floor_plan', 'area', 'charge', 'access1_route', 'access1_nearest_station', 'access1_walking_minutes', 'access2_route', 'access2_nearest_station', 'access2_walking_minutes']
merged_df = merged_df[column_order]

# 重複データを削除する（指定された列が一致する場合のみ）
duplicates_columns = ['title', 'address', 'floor', 'floor_plan', 'area']
merged_df.drop_duplicates(subset=duplicates_columns, inplace=True)

# 重複削除後の件数を確認
final_count = len(merged_df)
duplicates_count = pre_merge_count - final_count
print(f"重複削除件数: {duplicates_count}")

# SQLiteデータベースに接続
conn = sqlite3.connect('example.db')

# データフレームをSQLiteデータベースに格納
merged_df.to_sql('merged_data', conn, if_exists='replace', index=False)

# データベース接続を閉じる
conn.close()

# マージされたデータフレームの最初の5行を表示
print(merged_df.head())

重複削除件数: 2027
                  title      address age floor     fee management_fee deposit  \
0                   奥山方  東京都荒川区西日暮里２  41     1  150000              0  150000   
1  都営大江戸線 東中野駅 3階建 築21年   東京都新宿区北新宿３  21     1  190000          10000       0   
2              上中里2丁目戸建    東京都北区上中里２  47   1-3  135000              0  135000   
3              SORA赤塚新町  東京都板橋区赤塚新町１  63   1-2  130000              0  130000   
4              レジディア秋葉原    東京都台東区上野５  17    11  133000          10000  133000   

  gratuity floor_plan   area charge access1_route access1_nearest_station  \
0   150000         4K  73.75    NaN         ＪＲ山手線                    日暮里駅   
1   190000        4SK   80.0    NaN        都営大江戸線                    東中野駅   
2   135000         4K   60.0    NaN         ＪＲ高崎線                     尾久駅   
3   130000         5K   68.0    NaN         東武東上線                    下赤塚駅   
4   133000         2K  30.56    NaN         ＪＲ山手線                    秋葉原駅   

  access1_walking_minutes access2_rou